# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np

In [2]:
!python create_tables.py

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
cur.execute("""CREATE EXTENSION "uuid-ossp";""")

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
!ls

create_tables.py  etl.ipynb  __pycache__  sql_queries.py
data		  etl.py     README.md	  test.ipynb


In [6]:
song_files = get_files('data/song_data/')

In [7]:
filepath = song_files[0]

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

artist_id  artist_latitude artist_location  artist_longitude  \
0  AR8IEZO1187B99055E              NaN                               NaN   

    artist_name   duration  num_songs             song_id          title  year  
0  Marc Shaiman  149.86404          1  SOINLJW12A8C13314C  City Slickers  2008

## #1: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df.loc[0, ['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()
artist_data

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


['AR8IEZO1187B99055E', 'Marc Shaiman', nan, nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
artist_id, artist_latitude, artist_location, artist_longitude, artist_name, duration, num_songs, song_id, title, year = df.values[0]
song_data = [song_id, title, artist_id, year, duration] #
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
song_table_insert

'\nINSERT INTO songs\n(song_id, title, artist_id, year, duration)\nVALUES (%s, %s, %s, %s, %s)\nON CONFLICT(song_id) DO NOTHING\n'

In [13]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files('data/log_data')

In [15]:
filepath = log_files[0]

In [16]:
df = pd.read_json(filepath, lines=True)
df.head()

artist       auth firstName gender  itemInSession lastName  \
0  Sydney Youngblood  Logged In     Jacob      M             53    Klein   
1         Gang Starr  Logged In     Layla      F             88  Griffin   
2              3OH!3  Logged In     Layla      F             89  Griffin   
3        RÃÂ¶yksopp  Logged In     Jacob      M             54    Klein   
4         Kajagoogoo  Logged In     Layla      F             90  Griffin   

      length level                             location method      page  \
0  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
1  151.92771  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
2  192.52200  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
3  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  223.55546  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   

   registration  sessionId                                         song  \
0  1.540558e+12        954                            Ain't No Sunshine   
1  1.541057e+12        984                   My Advice 2 You (Explicit)   
2  1.541057e+12        984  My First Kiss (Feat. Ke$ha) [Album Version]   
3  1.540558e+12        954                       The Girl and The Robot   
4  1.541057e+12        984                                      Too Shy   

   status             ts                                          userAgent  \
0     200  1543449657796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
1     200  1543449690796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
2     200  1543449841796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
3     200  1543449895796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
4     200  1543450033796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   

  userId  
0     73  
1     24  
2     24  
3     73  
4     24

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df = df.loc[df.page=='NextSong']
df.head()

artist       auth firstName gender  itemInSession lastName  \
0  Sydney Youngblood  Logged In     Jacob      M             53    Klein   
1         Gang Starr  Logged In     Layla      F             88  Griffin   
2              3OH!3  Logged In     Layla      F             89  Griffin   
3        RÃÂ¶yksopp  Logged In     Jacob      M             54    Klein   
4         Kajagoogoo  Logged In     Layla      F             90  Griffin   

      length level                             location method      page  \
0  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
1  151.92771  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
2  192.52200  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
3  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  223.55546  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   

   registration  sessionId                                         song  \
0  1.540558e+12        954                            Ain't No Sunshine   
1  1.541057e+12        984                   My Advice 2 You (Explicit)   
2  1.541057e+12        984  My First Kiss (Feat. Ke$ha) [Album Version]   
3  1.540558e+12        954                       The Girl and The Robot   
4  1.541057e+12        984                                      Too Shy   

   status             ts                                          userAgent  \
0     200  1543449657796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
1     200  1543449690796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
2     200  1543449841796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
3     200  1543449895796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
4     200  1543450033796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   

  userId  
0     73  
1     24  
2     24  
3     73  
4     24

In [18]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [19]:
time_data = [t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ['timestamp', 'hour','day','week','month','year','weekday']

In [20]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

timestamp  hour  day  week  month  year  weekday
0 2018-11-29 00:00:57.796     0   29    48     11  2018        3
1 2018-11-29 00:01:30.796     0   29    48     11  2018        3
2 2018-11-29 00:04:01.796     0   29    48     11  2018        3
3 2018-11-29 00:04:55.796     0   29    48     11  2018        3
4 2018-11-29 00:07:13.796     0   29    48     11  2018        3

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df.loc[:,['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
df

artist       auth firstName gender  \
0                 Sydney Youngblood  Logged In     Jacob      M   
1                        Gang Starr  Logged In     Layla      F   
2                             3OH!3  Logged In     Layla      F   
3                       RÃÂ¶yksopp  Logged In     Jacob      M   
4                        Kajagoogoo  Logged In     Layla      F   
5            The Jimmy Castor Bunch  Logged In     Layla      F   
6                          Ministry  Logged In     Jacob      M   
7                    Bouncing Souls  Logged In     Jacob      M   
8                    Tegan And Sara  Logged In     Layla      F   
9                       Silverchair  Logged In     Layla      F   
10                           Enigma  Logged In     Layla      F   
11                             Anis  Logged In     Layla      F   
12                             Seal  Logged In     Layla      F   
13           UNKLE Feat. Josh Homme  Logged In     Layla      F   
14                    Dwight Yoakam  Logged In     Layla      F   
15                         Elastica  Logged In     Layla      F   
16                 Capone-N-Noreaga  Logged In     Layla      F   
17                       Morris Day  Logged In     Layla      F   
18                     Foo Fighters  Logged In     Layla      F   
19                            Olive  Logged In     Layla      F   
20                   Olle Adolphson  Logged In     Layla      F   
21            Streetlight Manifesto  Logged In       Ava      F   
22                      Frank Reyes  Logged In     Layla      F   
23                  Andres Calamaro  Logged In     Layla      F   
24                           Zero 7  Logged In     Layla      F   
25    Mark Ronson featuring Tiggers  Logged In     Layla      F   
26                      Keith Sweat  Logged In     Layla      F   
27                     Wakey!Wakey!  Logged In     Layla      F   
28              The Mercury Program  Logged In     Layla      F   
29   Kid Cudi / Kanye West / Common  Logged In     Layla      F   
..                              ...        ...       ...    ...   
333                Mission Of Burma  Logged In     Tegan      F   
334                     Danity Kane  Logged In     Chloe      F   
336                       Radiohead  Logged In     Tegan      F   
337                       Fightstar  Logged In     Wyatt      M   
338                      Hoobastank  Logged In     Tegan      F   
339                     Bright Eyes  Logged In     Tegan      F   
340                    Procol Harum  Logged In     Tegan      F   
341                       Radiohead  Logged In     Tegan      F   
342                 Alliance Ethnik  Logged In     Tegan      F   
343                            Muse  Logged In     Tegan      F   
344                           Steps  Logged In     Tegan      F   
345                      John Mayer  Logged In     Tegan      F   
346                   Kings Of Leon  Logged In      Sean      F   
347                      David Gray  Logged In     Tegan      F   
348                      King Stitt  Logged In      Sean      F   
349                        Tub Ring  Logged In     Tegan      F   
350                             Era  Logged In     Tegan      F   
352              + / - {Plus/Minus}  Logged In     Tegan      F   
353                 Crystal Castles  Logged In     Tegan      F   
354           The Whitest Boy Alive  Logged In     Tegan      F   
355               The White Stripes  Logged In     Tegan      F   
356                     Furthermore  Logged In     Tegan      F   
357                Eminem / Bizarre  Logged In     Tegan      F   
358          Florence + The Machine  Logged In     Tegan      F   
359                     Soda Stereo  Logged In    Braden      M   
360                    Taylor Swift  Logged In     Tegan      F   
362                       Pearl Jam  Logged In     Tegan      F   
363                 The 2 Live Crew  Logged In     Tegan      F   
364      

In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    id_name = (str(row.song) + str(row.ts) + str(row.userId),)
    generate_uuid = ("""SELECT uuid_generate_v5(uuid_nil(), %s)""")
    cur.execute(generate_uuid, id_name)
    songplay_generated_id = cur.fetchone() 

    # insert songplay record
    songplay_data = (songplay_generated_id, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [26]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [2]:
!python create_tables.py

In [3]:
!python etl.py

73 files found in data/song_data
1/73 files processed.
2/73 files processed.
3/73 files processed.
4/73 files processed.
5/73 files processed.
6/73 files processed.
7/73 files processed.
8/73 files processed.
9/73 files processed.
10/73 files processed.
11/73 files processed.
12/73 files processed.
13/73 files processed.
14/73 files processed.
15/73 files processed.
16/73 files processed.
17/73 files processed.
18/73 files processed.
19/73 files processed.
20/73 files processed.
21/73 files processed.
22/73 files processed.
23/73 files processed.
24/73 files processed.
25/73 files processed.
26/73 files processed.
27/73 files processed.
28/73 files processed.
29/73 files processed.
30/73 files processed.
31/73 files processed.
32/73 files processed.
33/73 files processed.
34/73 files processed.
35/73 files processed.
36/73 files processed.
37/73 files processed.
38/73 files processed.
39/73 files processed.
40/73 files processed.
41/73 files processed.
42/73 files processed.
43/73 file